#### Notes

The difference b/w this and the 14bins run is that we did a corresponding BBarolo run to better match our MCMC runs. The changes to the BBarolo run are:

- 28 bins
- 1 value of inc and pa for entire galaxy
- vdisp = 10
- better center: (511, 488) --> (511, 512)
- 2d fit

Because of this, the ringlog in the new run goes out only to ~ 5 kpc. We're fitting 31 bins here because we're fitting 28 bins out to the ringlog rmax and then 3 more bins to extend out and then another "infinite" bin to 10,000 kpc.

note: I haven't cropped the new 1st mom map yet


#### using 2DFIT task from BBarolo to make it apples to apples

In [1]:
from fit2d import Galaxy, RingModel
from fit2d.mcmc import LinearPrior
from fit2d.mcmc import emcee_lnlike, piecewise_start_points
from fit2d.models import PiecewiseModel

from astropy.io import fits
import copy
from datetime import datetime
import glob
import joblib
import numpy as np
import matplotlib.pyplot as plt
import os

import emcee
from emcee import EnsembleSampler, moves

In [2]:
num_bins = 31
# min and max bounds for each bin
vmin, vmax = 0, 200

# min and max inc, pos angle in RADIANS
inc_min, inc_max = np.radians(45), np.radians(75)
pos_angle_min, pos_angle_max = np.radians(0), np.radians(360)

name = "NGC2366"
distance = 3400. # [kpc]

home_directory = "/Users/stephencoffey/Downloads/little-things/BBarolo_runs/2D_output"
observed_2d_vel_field_fits_file = f"{home_directory}/{name}/{name}map_1st.fits"
observed_2d_intensity_field_fits_file = "/Users/stephencoffey/Downloads/little-things/BBarolo_runs/3D_output/NGC2366_ftype1/maps/NGC2366_0mom.fits"

# to use the dispersion as a source of errors, provide this file name
# observed_2d_dispersion_fits_file = f"{home_directory}/NGC2366_2mom.fits"
deg_per_pixel=4.17e-4

ring_param_file = f"{home_directory}/{name}/{name}_2dtrm.txt"
v_systemic = 100 #changed to 100 to equal exactly what we used in mathematica

In [3]:
# x and y dims are switched in ds9 fits display versus np array shape
fits_ydim, fits_xdim = fits.open(observed_2d_vel_field_fits_file)[0].data.shape

mask_sigma=1.
random_seed = 1234

mcmc_nwalkers = 70
mcmc_niter = 1000
mcmc_ndim = num_bins + 2  # Do not change this if fitting one ring at a time
mcmc_nthreads = 40
# Try increasing stretch scale factor a. version must be >=3 for this to be used.
mcmc_moves = moves.StretchMove(a = 2)
mcmc_version = float(emcee.__version__[0])

# Option to save every batch_size iterations in case of crash<br>
# Increase this; 2 is a very low value just for testing

batch_size = 50

# option to save outputs in a particular directory
save_dir = "/Users/stephencoffey/Downloads/little-things/mcmc_pickles/bs_31bins/"

In [4]:
galaxy = Galaxy(
    name=name,
    distance=distance,
    observed_2d_vel_field_fits_file=observed_2d_vel_field_fits_file,
    deg_per_pixel=deg_per_pixel,
    v_systemic=v_systemic,
    observed_2d_intensity_field_fits_file=observed_2d_intensity_field_fits_file
)

In [5]:
ring_model = RingModel(
    ring_param_file=ring_param_file,
    fits_xdim=fits_xdim,
    fits_ydim=fits_ydim,
    distance=distance
)

ring_param_bounds = [(vmin, vmax)] * num_bins + [(inc_min, inc_max), (pos_angle_min, pos_angle_max)]

In [6]:
fit_structural_params = {"inc": -2, "pos_angle": -1}

In [7]:
vels, pos_angles, incs = np.loadtxt(ring_param_file, usecols=(3, 5, 6)).T
radsep = ring_model.radii_kpc[-1] - ring_model.radii_kpc[-2]

bin_edges = [0 + i*radsep for i in range(len(ring_model.radii_kpc)+1)]
bin_edges = np.append(bin_edges, np.linspace(bin_edges[-1], 2 * bin_edges[-1], 4)[-3:])
bin_edges = np.append(bin_edges, 10000)
v_rot = np.append(vels, [vels[-1] for i in range(4)])
outer_bin_centers = [np.mean([bin_edges[i], bin_edges[i+1]]) for i in range(27,31)]

In [8]:
v_err_const = 10. # [km/s] constant error per pixel
v_err_2d = None
#v_err_2d = galaxy.observed_2d_dispersion

In [9]:
import warnings
warnings.simplefilter('ignore')
from fit2d._velocity_field_generator import create_2d_velocity_field

inc = np.radians(incs[0])  # grabbing one point in ringlog, since in BB it's const
pos_angle = np.radians(pos_angles[0]) # grabbing one point in ringlog, since in BB it's const

# if inc and/or pos_angle are not being fit in the MCMC, they will be fixed to constant values inc_fake, pos_angle_fake
ring_model.update_structural_parameters(inc=inc, pos_angle=pos_angle)

In [10]:
# creating the starting position bounds
bounds = []
for i in range(len(v_rot)):
    bounds.append((v_rot[i] - 1, v_rot[i] + 1))
bounds.extend([(inc - 0.1, inc + 0.1), (pos_angle - 0.1, pos_angle + 0.1)]) # only use if fitting inc and pa

In [11]:
from fit2d.mcmc._likelihood import chisq_2d, lnlike

bin_min, bin_max = bin_edges[0], bin_edges[-1]

# 1st moment map
galaxy.observed_2d_vel_field = fits.open(observed_2d_vel_field_fits_file)[0].data
mask = np.nan_to_num(galaxy.observed_2d_vel_field/galaxy.observed_2d_vel_field, nan=0.)
dimensions = galaxy.observed_2d_vel_field.shape

# 0th moment map
observed_2d_intensity = fits.open(observed_2d_intensity_field_fits_file)[0].data


piecewise_model = PiecewiseModel(num_bins=num_bins)
piecewise_model.set_bounds(array_bounds=ring_param_bounds)
#piecewise_model.set_bin_edges(rmin=bin_min, rmax=bin_max)
piecewise_model.bin_edges = bin_edges
radii_to_interpolate = np.append(ring_model.radii_kpc, outer_bin_centers) # manually gave bin centers to be the BB values + 4 outer bin centers
#print("bin centers:", radii_to_interpolate)
#radii_to_interpolate = np.array([r for r in bin_edges if bin_min<=r<=bin_max])
prior = LinearPrior(bounds=piecewise_model.bounds)
prior_transform = prior.transform_from_unit_cube
# instead of using piecewise_model.bounds, we've manually input bounds for the starting positions so the walkers start out much closer to the value we're looking for
start_positions = piecewise_start_points(mcmc_nwalkers, bounds = bounds, random_seed=random_seed)
fit_inputs = {
    "piecewise_model": piecewise_model,
    "galaxy": galaxy,
    "ring_model": ring_model,
    "prior_transform": prior_transform
}


rotation_curve_func_kwargs = {
    "radii_to_interpolate": radii_to_interpolate}
lnlike_args = {
    "model": piecewise_model,
    "rotation_curve_func_kwargs": rotation_curve_func_kwargs,
    "galaxy": galaxy,
    "ring_model": ring_model,
    "mask_sigma": mask_sigma,
    "v_err_const": v_err_const,
    "v_err_2d": v_err_2d,
    "fit_structural_params": fit_structural_params
}

sampler = EnsembleSampler(
    mcmc_nwalkers,
    mcmc_ndim,
    emcee_lnlike,
    args=[mcmc_version, lnlike_args],
    threads=mcmc_nthreads,
)
if mcmc_version >= 3:
    sampler._moves = [mcmc_moves]
sampler_output_file = os.path.join(
    save_dir or "", f"sampler_{galaxy.name}_{mcmc_niter}iter.pkl")

In [12]:
for batch in range(mcmc_niter // batch_size):
    if batch == 0:
        batch_start = start_positions
    else:
        batch_start = None
        sampler.pool = temp_pool
    sampler.run_mcmc(batch_start, batch_size)
    temp_pool = sampler.pool
    del sampler.pool
    with open(sampler_output_file, 'wb') as f:
        sampler_copy = copy.copy(sampler)
        del sampler_copy.log_prob_fn
        joblib.dump(sampler_copy, f)

    print(f"Done with steps {batch*batch_size} - {(batch+1)*batch_size} out of {mcmc_niter}")

Done with steps 0 - 50 out of 1000
Done with steps 50 - 100 out of 1000
Done with steps 100 - 150 out of 1000
Done with steps 150 - 200 out of 1000
Done with steps 200 - 250 out of 1000
Done with steps 250 - 300 out of 1000
Done with steps 300 - 350 out of 1000
Done with steps 350 - 400 out of 1000
Done with steps 400 - 450 out of 1000
Done with steps 450 - 500 out of 1000
Done with steps 500 - 550 out of 1000
Done with steps 550 - 600 out of 1000
Done with steps 600 - 650 out of 1000
Done with steps 650 - 700 out of 1000
Done with steps 700 - 750 out of 1000
Done with steps 750 - 800 out of 1000
Done with steps 800 - 850 out of 1000
Done with steps 850 - 900 out of 1000
Done with steps 900 - 950 out of 1000
Done with steps 950 - 1000 out of 1000


In [13]:
print(f"Done with emcee fit for {galaxy.name}")

Done with emcee fit for NGC2366


In [14]:
"""log_probs = sampler.get_log_prob()
print("log prob of best fit point:", np.amax(log_probs))
log_max = np.where(log_probs == np.amax(log_probs))
param_vals = sampler.get_chain()[log_max]
print("vel, inc, vsini of best fit point:", (param_vals[0][0], param_vals[0][1],param_vals[0][0]*np.sin(param_vals[0][1]) ))
print("Mean acceptance fraction: {0:.3f}".format(np.mean(sampler.acceptance_fraction)))
#print("Mean autocorrelation time: {0:.3f} steps".format(np.mean(sampler.get_autocorr_time())))"""

'log_probs = sampler.get_log_prob()\nprint("log prob of best fit point:", np.amax(log_probs))\nlog_max = np.where(log_probs == np.amax(log_probs))\nparam_vals = sampler.get_chain()[log_max]\nprint("vel, inc, vsini of best fit point:", (param_vals[0][0], param_vals[0][1],param_vals[0][0]*np.sin(param_vals[0][1]) ))\nprint("Mean acceptance fraction: {0:.3f}".format(np.mean(sampler.acceptance_fraction)))\n#print("Mean autocorrelation time: {0:.3f} steps".format(np.mean(sampler.get_autocorr_time())))'